In [16]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [17]:
import pandas as pd 
import numpy as np
from pytictoc import TicToc
import warnings

In [18]:
from typing import Union, Dict, List, Tuple, Any, Optional, Callable
import warnings
import logging
import sys
import numpy as np
import pandas as pd
import sklearn
import sklearn.pipeline
from sklearn.base import clone
import inspect
from copy import copy

import skforecast
from skforecast.utils import initialize_lags
from skforecast.utils import initialize_weights
from skforecast.utils import check_select_fit_kwargs
from skforecast.utils import check_y
from skforecast.utils import check_exog
from skforecast.utils import get_exog_dtypes
from skforecast.utils import check_exog_dtypes
from skforecast.utils import check_predict_input
from skforecast.utils import check_interval
from skforecast.utils import preprocess_y
from skforecast.utils import preprocess_last_window
from skforecast.utils import preprocess_exog
from skforecast.utils import exog_to_direct
from skforecast.utils import expand_index
from skforecast.utils import transform_series
from skforecast.utils import transform_dataframe
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.linear_model import LinearRegression

In [19]:
# Downloading data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-'
       'learning-python/master/data/bike_sharing_dataset_clean.csv')
data = pd.read_csv(url)

# Preprocess data
# ==============================================================================
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
data = data.set_index('date_time')
data = data.asfreq('H')
data = data.sort_index()
data['holiday'] = data['holiday'].astype(int)
data = data[['holiday', 'weather', 'temp', 'hum', 'users']]
data[['holiday', 'weather']] = data[['holiday', 'weather']].astype(str)
print(data.dtypes)
data.head(3)
# Split train-test
# ==============================================================================
start_train = '2012-06-01 00:00:00'
end_train = '2012-07-31 23:59:00'
end_test = '2012-08-15 23:59:00'
data_train = data.loc[start_train:end_train, :]
data_test  = data.loc[end_train:end_test, :]

holiday     object
weather     object
temp       float64
hum        float64
users      float64
dtype: object


In [20]:
steps=200
forecaster = ForecasterAutoregDirect(
                regressor = LinearRegression(),
                lags = 150,
                steps = steps,
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
)

forecaster.predict(steps=steps)

2012-08-01 00:00:00     14.064264
2012-08-01 01:00:00    -10.434793
2012-08-01 02:00:00     -7.650264
2012-08-01 03:00:00      6.128299
2012-08-01 04:00:00    -13.116071
                          ...    
2012-08-09 03:00:00    -13.832459
2012-08-09 04:00:00    -29.158999
2012-08-09 05:00:00      9.914552
2012-08-09 06:00:00    160.762730
2012-08-09 07:00:00    474.384955
Name: pred, Length: 200, dtype: float64

In [21]:
def predict(
    self,
    steps: Optional[Union[int, list]]=None,
    last_window: Optional[pd.Series]=None,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None
) -> pd.Series:
    """
    Predict n steps ahead.

    Parameters
    ----------
    steps : int, list, None, default `None`
        Predict n steps. The value of `steps` must be less than or equal to the 
        value of steps defined when initializing the forecaster. Starts at 1.
    
        If `int`:
            Only steps within the range of 1 to int are predicted.
    
        If `list`:
            List of ints. Only the steps contained in the list are predicted.

        If `None`:
            As many steps are predicted as were defined at initialization.

    last_window : pandas Series, default `None`
        Series values used to create the predictors (lags) needed in the 
        first iteration of the prediction (t + 1).

        If `last_window = None`, the values stored in` self.last_window` are
        used to calculate the initial predictors, and the predictions start
        right after training data.
        
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s.

    Returns 
    -------
    predictions : pandas Series
        Predicted values.

    """

    if isinstance(steps, int):
        steps = list(np.arange(steps) + 1)
    elif steps is None:
        steps = list(np.arange(self.steps) + 1)
    elif isinstance(steps, list):
        steps = list(np.array(steps))

    for step in steps:
        if not isinstance(step, (int, np.int64, np.int32)):
            raise TypeError(
                (f"`steps` argument must be an int, a list of ints or `None`. "
                    f"Got {type(steps)}.")
            )

    if last_window is None:
        last_window = copy(self.last_window)

    check_predict_input(
        forecaster_name  = type(self).__name__,
        steps            = steps,
        fitted           = self.fitted,
        included_exog    = self.included_exog,
        index_type       = self.index_type,
        index_freq       = self.index_freq,
        window_size      = self.window_size,
        last_window      = last_window,
        last_window_exog = None,
        exog             = exog,
        exog_type        = self.exog_type,
        exog_col_names   = self.exog_col_names,
        interval         = None,
        alpha            = None,
        max_steps        = self.steps,
        levels           = None,
        series_col_names = None
    ) 

    if exog is not None:
        if isinstance(exog, pd.DataFrame):
            exog = transform_dataframe(
                        df                = exog,
                        transformer       = self.transformer_exog,
                        fit               = False,
                        inverse_transform = False
                    )
        else:
            exog = transform_series(
                        series            = exog,
                        transformer       = self.transformer_exog,
                        fit               = False,
                        inverse_transform = False
                    )
        check_exog_dtypes(exog=exog)
        exog_values = exog_to_direct(exog=exog.iloc[:max(steps), ], steps=max(steps)).to_numpy()
    else:
        exog_values = None

    last_window = transform_series(
                        series            = last_window,
                        transformer       = self.transformer_y,
                        fit               = False,
                        inverse_transform = False
                    )
    last_window_values, last_window_index = preprocess_last_window(
                                                last_window = last_window
                                            )

    X_lags = last_window_values[-self.lags].reshape(1, -1)
    
    predictions = np.full(shape=len(steps), fill_value=np.nan)

    for i, step in enumerate(steps):
        regressor = self.regressors_[step]
        if exog is None:
            X = X_lags
        else:
            # Only columns from exog related with the current step are selected.
            X = np.hstack([X_lags, exog_values[0][step-1::max(steps)].reshape(1, -1)])
        with warnings.catch_warnings():
            # Suppress scikit-learn warning: "X does not have valid feature names,
            # but NoOpTransformer was fitted with feature names".
            warnings.simplefilter("ignore")
            predictions[i] = regressor.predict(X)

    idx = expand_index(index=last_window_index, steps=max(steps))

    predictions = pd.Series(
                        data  = predictions.reshape(-1),
                        index = idx[np.array(steps)-1],
                        name  = 'pred'
                    )

    predictions = transform_series(
                        series            = predictions,
                        transformer       = self.transformer_y,
                        fit               = False,
                        inverse_transform = True
                    )

    return predictions

In [22]:
def predict_numpy(
    self,
    steps: Optional[Union[int, list]]=None,
    last_window: Optional[pd.Series]=None,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None
) -> pd.Series:
    """
    Predict n steps ahead.

    Parameters
    ----------
    steps : int, list, None, default `None`
        Predict n steps. The value of `steps` must be less than or equal to the 
        value of steps defined when initializing the forecaster. Starts at 1.
    
        If `int`:
            Only steps within the range of 1 to int are predicted.
    
        If `list`:
            List of ints. Only the steps contained in the list are predicted.

        If `None`:
            As many steps are predicted as were defined at initialization.

    last_window : pandas Series, default `None`
        Series values used to create the predictors (lags) needed in the 
        first iteration of the prediction (t + 1).

        If `last_window = None`, the values stored in` self.last_window` are
        used to calculate the initial predictors, and the predictions start
        right after training data.
        
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s.

    Returns 
    -------
    predictions : pandas Series
        Predicted values.

    """

    if isinstance(steps, int):
        steps = list(np.arange(steps) + 1)
    elif steps is None:
        steps = list(np.arange(self.steps) + 1)
    elif isinstance(steps, list):
        steps = list(np.array(steps))

    for step in steps:
        if not isinstance(step, (int, np.int64, np.int32)):
            raise TypeError(
                (f"`steps` argument must be an int, a list of ints or `None`. "
                    f"Got {type(steps)}.")
            )

    if last_window is None:
        last_window = copy(self.last_window)

    check_predict_input(
        forecaster_name  = type(self).__name__,
        steps            = steps,
        fitted           = self.fitted,
        included_exog    = self.included_exog,
        index_type       = self.index_type,
        index_freq       = self.index_freq,
        window_size      = self.window_size,
        last_window      = last_window,
        last_window_exog = None,
        exog             = exog,
        exog_type        = self.exog_type,
        exog_col_names   = self.exog_col_names,
        interval         = None,
        alpha            = None,
        max_steps        = self.steps,
        levels           = None,
        series_col_names = None
    ) 

    if exog is not None:
        if isinstance(exog, pd.DataFrame):
            exog = transform_dataframe(
                        df                = exog,
                        transformer       = self.transformer_exog,
                        fit               = False,
                        inverse_transform = False
                    )
        else:
            exog = transform_series(
                        series            = exog,
                        transformer       = self.transformer_exog,
                        fit               = False,
                        inverse_transform = False
                    )
        check_exog_dtypes(exog=exog)
        exog_values = exog_to_direct(exog=exog.iloc[:max(steps), ], steps=max(steps)).to_numpy()
    else:
        exog_values = None

    last_window = transform_series(
                        series            = last_window,
                        transformer       = self.transformer_y,
                        fit               = False,
                        inverse_transform = False
                    )
    last_window_values, last_window_index = preprocess_last_window(
                                                last_window = last_window
                                            )

    X_lags = last_window_values[-self.lags].reshape(1, -1)
    
    predictions = np.full(shape=len(steps), fill_value=np.nan)

    if exog is None:
        Xs = [X_lags] * len(steps)
    else:
        Xs = [
            np.hstack([X_lags, exog_values[0][step-1::max(steps)].reshape(1, -1)])
            for step in steps
        ]

    with warnings.catch_warnings():
            # Suppress scikit-learn warning: "X does not have valid feature names,
            # but NoOpTransformer was fitted with feature names".
            warnings.simplefilter("ignore")
            predictions = [
                regressor.predict(X) for regressor, X in zip(self.regressors_.values(), Xs)
            ]

    idx = expand_index(index=last_window_index, steps=max(steps))
    predictions = pd.Series(
                        data  = predictions,
                        index = idx[np.array(steps)-1],
                        name  = 'pred'
                    )

    predictions = transform_series(
                        series            = predictions,
                        transformer       = self.transformer_y,
                        fit               = False,
                        inverse_transform = True
                    )

    return predictions

In [23]:
def predict_pandas(
    self,
    steps: Optional[Union[int, list]]=None,
    last_window: Optional[pd.Series]=None,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None
) -> pd.Series:
    """
    Predict n steps ahead.

    Parameters
    ----------
    steps : int, list, None, default `None`
        Predict n steps. The value of `steps` must be less than or equal to the 
        value of steps defined when initializing the forecaster. Starts at 1.
    
        If `int`:
            Only steps within the range of 1 to int are predicted.
    
        If `list`:
            List of ints. Only the steps contained in the list are predicted.

        If `None`:
            As many steps are predicted as were defined at initialization.

    last_window : pandas Series, default `None`
        Series values used to create the predictors (lags) needed in the 
        first iteration of the prediction (t + 1).

        If `last_window = None`, the values stored in` self.last_window` are
        used to calculate the initial predictors, and the predictions start
        right after training data.
        
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s.

    Returns 
    -------
    predictions : pandas Series
        Predicted values.

    """

    if isinstance(steps, int):
        steps = list(np.arange(steps) + 1)
    elif steps is None:
        steps = list(np.arange(self.steps) + 1)
    elif isinstance(steps, list):
        steps = list(np.array(steps))

    for step in steps:
        if not isinstance(step, (int, np.int64, np.int32)):
            raise TypeError(
                (f"`steps` argument must be an int, a list of ints or `None`. "
                    f"Got {type(steps)}.")
            )

    if last_window is None:
        last_window = copy(self.last_window)

    _, last_window_index = preprocess_last_window(
                                last_window   = last_window,
                                return_values = False
                           )
    idx = expand_index(index=last_window_index, steps=max(steps))
    predictions = pd.Series(
                        data  = np.nan,
                        index = idx[np.array(steps)-1],
                        name  = 'pred'
                    )
    X_lags = last_window.iloc[-self.lags]
    X_lags.index = [f"lag_{lag}" for lag in self.lags]
    X_lags = X_lags.to_frame().T

    if exog is None:
        Xs = [X_lags] * len(steps)
    else:
        Xs = [
            pd.concat([X_lags, exog.iloc[step-1::max(steps)]], axis=1)
            for step in steps
        ]

    predictions = [regressor.predict(X) for regressor, X in zip(self.regressors_.values(), Xs)]
    predictions = pd.Series(
                    data  = predictions,
                    index = idx[np.array(steps)-1],
                    name  = 'pred',
                  )

    return predictions

In [24]:
def predict_pandas_2(
    self,
    steps: Optional[Union[int, list]]=None,
    last_window: Optional[pd.Series]=None,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None
) -> pd.Series:
    """
    Predict n steps ahead.

    Parameters
    ----------
    steps : int, list, None, default `None`
        Predict n steps. The value of `steps` must be less than or equal to the 
        value of steps defined when initializing the forecaster. Starts at 1.
    
        If `int`:
            Only steps within the range of 1 to int are predicted.
    
        If `list`:
            List of ints. Only the steps contained in the list are predicted.

        If `None`:
            As many steps are predicted as were defined at initialization.

    last_window : pandas Series, default `None`
        Series values used to create the predictors (lags) needed in the 
        first iteration of the prediction (t + 1).

        If `last_window = None`, the values stored in` self.last_window` are
        used to calculate the initial predictors, and the predictions start
        right after training data.
        
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s.

    Returns 
    -------
    predictions : pandas Series
        Predicted values.

    """

    if isinstance(steps, int):
        steps = list(np.arange(steps) + 1)
    elif steps is None:
        steps = list(np.arange(self.steps) + 1)
    elif isinstance(steps, list):
        steps = list(np.array(steps))

    for step in steps:
        if not isinstance(step, (int, np.int64, np.int32)):
            raise TypeError(
                (f"`steps` argument must be an int, a list of ints or `None`. "
                    f"Got {type(steps)}.")
            )

    if last_window is None:
        last_window = copy(self.last_window)

    _, last_window_index = preprocess_last_window(
                                last_window   = last_window,
                                return_values = False
                           )
    idx = expand_index(index=last_window_index, steps=max(steps))
    predictions = pd.Series(
                        data  = np.nan,
                        index = idx[np.array(steps)-1],
                        name  = 'pred'
                    )
    X_lags = pd.DataFrame(
                data    = last_window.values[-self.lags].reshape(1, -1),
                columns = [f"lag_{lag}" for lag in self.lags]
            )

    if exog is None:
        Xs = [X_lags] * len(steps)
    else:
        Xs = [
            pd.concat([X_lags, exog.iloc[step-1::max(steps)]], axis=1)
            for step in steps
        ]

    with warnings.catch_warnings():
        # Suppress scikit-learn warning: "X does not have valid feature names,
        # but NoOpTransformer was fitted with feature names".
        warnings.simplefilter("ignore")
        predictions = [
            regressor.predict(X) for regressor, X in zip(self.regressors_.values(), Xs)
        ]
        
    predictions = pd.Series(
                    data  = predictions,
                    index = idx[np.array(steps)-1],
                    name  = 'pred',
                  )

    return predictions

In [25]:
forecaster
expected = forecaster.predict(steps=steps)
print(expected)

2012-08-01 00:00:00     14.064264
2012-08-01 01:00:00    -10.434793
2012-08-01 02:00:00     -7.650264
2012-08-01 03:00:00      6.128299
2012-08-01 04:00:00    -13.116071
                          ...    
2012-08-09 03:00:00    -13.832459
2012-08-09 04:00:00    -29.158999
2012-08-09 05:00:00      9.914552
2012-08-09 06:00:00    160.762730
2012-08-09 07:00:00    474.384955
Name: pred, Length: 200, dtype: float64


In [26]:
t = TicToc()
t.tic()
forecaster.predict(steps=steps)
t.toc()


Elapsed time is 0.013528 seconds.


In [27]:
t = TicToc()
t.tic()
predict(self=forecaster, steps=steps)
t.toc()
pd.testing.assert_series_equal(expected, predict(self=forecaster, steps=steps))

Elapsed time is 0.011639 seconds.


In [28]:
t = TicToc()
t.tic()
predict_numpy(self=forecaster, steps=steps)
t.toc()
pd.testing.assert_series_equal(expected, predict(self=forecaster, steps=steps))

Elapsed time is 0.009455 seconds.


In [29]:
t = TicToc()
t.tic()
predict_pandas(self=forecaster, steps=steps)
t.toc()
pd.testing.assert_series_equal(expected, predict(self=forecaster, steps=steps))

Elapsed time is 0.251986 seconds.


In [30]:
t = TicToc()
t.tic()
predict_pandas_2(self=forecaster, steps=steps)
t.toc()
pd.testing.assert_series_equal(expected, predict(self=forecaster, steps=steps))

Elapsed time is 0.253851 seconds.
